In [2]:
import findspark
findspark.init()
print(findspark.find())

/Users/bavisettisivaavinash/hadoopworker/spark-3.5.1-bin-hadoop3


In [ ]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("ReadExcel").getOrCreate()

# Read the Excel file using pandas
excel_df1 = pd.read_excel('Customer_Product_modified.xlsx')
excel_df2 = pd.read_excel('Customer_Transaction_history_modified.xlsx')
excel_df3 = pd.read_excel('Customer_Channel_Activity_modified.xlsx')
excel_df5 = pd.read_excel('Product_Lookup_modified.xlsx')


spark_df1 = spark.createDataFrame(excel_df1)
spark_df2 = spark.createDataFrame(excel_df2)
spark_df3 = spark.createDataFrame(excel_df3)
spark_df5 = spark.createDataFrame(excel_df5)

spark_df4 = spark.read.csv('customer_demographic_modified.xlsx - demographic.csv', header=True, inferSchema=True)

changed datatype

In [4]:
from pyspark.sql.functions import to_date, col

spark_df1 = spark_df1.withColumn('Start date', to_date(col('start date'))).withColumn('End date', to_date(col('End date')))

In [5]:
spark_df1.printSchema()

root
 |-- CSID: long (nullable = true)
 |-- Prod code: long (nullable = true)
 |-- Start date: date (nullable = true)
 |-- End date: date (nullable = true)



In [6]:
spark_df1.show()

+----------+---------+----------+----------+
|      CSID|Prod code|Start date|  End date|
+----------+---------+----------+----------+
|2345450601|     7001|2021-12-17|4399-12-31|
|2345450602|     3001|1996-10-29|4399-12-31|
|2345450603|     8004|2021-06-01|4399-12-31|
|2345450604|     7001|2020-08-12|4399-12-31|
|2345450605|     5004|2010-05-05|4399-12-31|
|2345450606|     8004|2017-06-19|4399-12-31|
|2345450607|     1004|2014-09-13|4399-12-31|
|2345450608|     2003|2023-06-10|4399-12-31|
|2345450609|     5002|2011-09-21|4399-12-31|
|2345450610|     2002|2020-03-09|4399-12-31|
|2345450611|     1006|1978-02-06|4399-12-31|
|2345450612|     3004|2020-05-20|4399-12-31|
|2345450613|     5001|2004-10-01|4399-12-31|
|2345450614|     8005|2022-12-07|4399-12-31|
|2345450615|     2005|2024-01-25|4399-12-31|
|2345450616|     2005|2022-12-15|4399-12-31|
|2345450617|     6003|2004-01-23|4399-12-31|
|2345450618|     3005|2009-03-04|4399-12-31|
|2345450619|     5002|2011-05-15|4399-12-31|
|234545062

In [7]:
spark_df2.printSchema()

root
 |-- CSID: long (nullable = true)
 |-- Trans_type: string (nullable = true)
 |-- Merchant_Category: string (nullable = true)
 |-- Merchant_Code: long (nullable = true)
 |-- Trans Amount: double (nullable = true)
 |-- Trans Date: timestamp (nullable = true)



In [8]:
spark_df2 = spark_df2.withColumn('Trans Date', to_date(col('Trans Date')))

In [9]:
spark_df2.show(5)

24/07/11 15:13:40 WARN TaskSetManager: Stage 3 contains a task of very large size (3278 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------------------+-----------------+-------------+------------+----------+
|      CSID|          Trans_type|Merchant_Category|Merchant_Code|Trans Amount|Trans Date|
+----------+--------------------+-----------------+-------------+------------+----------+
|2345450601|Regular Payment Card|    Miscellaneous|         8567|      714.44|2022-02-02|
|2345450602|Regular Payment Card|    Digital Goods|         8568|      434.93|1997-07-06|
|2345450603|Regular Payment Card| Food & Beverages|         8569|     1463.98|2022-08-24|
|2345450604|Regular Payment Card|         Airlines|         8570|     1367.74|2020-10-18|
|2345450605|          Debit Card|             fuel|         8571|      149.43|2010-07-15|
+----------+--------------------+-----------------+-------------+------------+----------+
only showing top 5 rows



24/07/11 15:13:44 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 3 (TID 5): Attempting to kill Python Worker


In [10]:
spark_df3.show(5)

24/07/11 15:13:44 WARN TaskSetManager: Stage 4 contains a task of very large size (1622 KiB). The maximum recommended task size is 1000 KiB.


+----------+---------+-------------------+---------------+
|      CSID|  Channel|       ActivityDate|   ActivityType|
+----------+---------+-------------------+---------------+
|2345450601|Telephone|2022-05-28 00:00:00|Account Opening|
|2345450611|      App|1987-05-21 00:00:00|Account Opening|
|2345450616|   Branch|2024-05-01 00:00:00|Account Opening|
|2345450635|      App|2022-01-17 00:00:00|Account Opening|
|2345450637|      App|2023-11-18 00:00:00|Account Opening|
+----------+---------+-------------------+---------------+
only showing top 5 rows



24/07/11 15:13:48 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 4 (TID 6): Attempting to kill Python Worker


In [11]:
spark_df3.printSchema()

root
 |-- CSID: long (nullable = true)
 |-- Channel: string (nullable = true)
 |-- ActivityDate: timestamp (nullable = true)
 |-- ActivityType: string (nullable = true)



In [12]:
from pyspark.sql.functions import date_format
spark_df3 = spark_df3.withColumn('ActivityDate', date_format(to_date(col('ActivityDate')), 'yyyy-MM-dd'))

In [13]:
spark_df3 = spark_df3.withColumn('ActivityDate', to_date(col('ActivityDate')))

In [14]:
spark_df4.show()

+----------+------+----------+------+----------+-----------------------+--------------+--------+
|      CSID|Income|dependants|Gender|Birth date|Relationship Start date|Marital_Status|Zip_Code|
+----------+------+----------+------+----------+-----------------------+--------------+--------+
|2345450601|200000|         3|     ?|  3/8/1976|               8/6/2017|      Divorced|CH42 0HS|
|2345450602|100000|         2|     ?|28-11-1963|               8/3/1983|      Divorced|ME20 6PR|
|2345450603|280000|         2|     ?|  5/6/1979|               1/9/2008|       Married|OX17 1EQ|
|2345450604|300000|         1|     ?|  9/5/1999|             10/12/2020|      Divorced| RH7 6LT|
|2345450605|150000|         1|  Male|31-03-2004|               6/3/2011|       Married| NW2 7RJ|
|2345450606|100000|         3|     ?|18-02-1987|             17-12-2002|        Single| TQ1 3PZ|
|2345450607|130000|         2|     M| 9/12/1961|             27-11-1983|      Divorced| LU6 3NP|
|2345450608|130000|         1|

In [15]:
spark_df4.printSchema()

root
 |-- CSID: long (nullable = true)
 |-- Income: integer (nullable = true)
 |-- dependants: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Birth date: string (nullable = true)
 |-- Relationship Start date: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Zip_Code: string (nullable = true)



In [16]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from datetime import datetime

def convert_date(date_str):
    for fmt in ('%d-%m-%Y', '%m/%d/%Y'):
        try:
            return datetime.strptime(date_str, fmt).strftime('%Y-%m-%d')
        except ValueError:
            pass
    return None

convert_date_udf = udf(convert_date, StringType())

spark_df4 = spark_df4.withColumn('Birth date', convert_date_udf(col('Birth date')))
spark_df4 = spark_df4.withColumn('Relationship Start date', convert_date_udf(col('Relationship Start date')))

In [17]:
from pyspark.sql.functions import when, col
spark_df4 = spark_df4.withColumn('Gender', when(col('Gender') == '?', 'unidentified').otherwise(col('Gender')))

In [18]:
from pyspark.sql.functions import regexp_replace


spark_df4 = spark_df4.withColumn('Zip_Code', regexp_replace(col('Zip_Code'), ' ', ''))

In [19]:
spark_df4 = spark_df4.withColumn('Birth date', to_date(col('Birth date'))).withColumn('Relationship Start date', to_date(col('Relationship Start date')))

In [20]:
spark_df5.printSchema()

root
 |-- Prod code: long (nullable = true)
 |-- Prod cls: string (nullable = true)
 |-- Prod name: string (nullable = true)
 |-- Start Date: timestamp (nullable = true)
 |-- End Date: timestamp (nullable = true)



In [21]:
spark_df5.show()

+---------+--------+-------------------+-------------------+-------------------+
|Prod code|Prod cls|          Prod name|         Start Date|           End Date|
+---------+--------+-------------------+-------------------+-------------------+
|     1001|   Loans|          Home loan|1980-01-01 00:00:00|2007-06-13 00:00:00|
|     1002|   Loans|      Personal Loan|1980-01-01 00:00:00|4599-12-31 00:00:00|
|     1003|   Loans|           car loan|2001-06-01 00:00:00|4599-12-31 00:00:00|
|     1004|   Loans|       student loan|2007-06-13 00:00:00|4599-12-31 00:00:00|
|     1005|   Loans|          gold loan|2010-05-01 00:00:00|4599-12-31 00:00:00|
|     1006|   Loans|    automobile loan|1975-06-01 00:00:00|4599-12-31 00:00:00|
|     1007|   Loans|      buisness loan|2000-05-01 00:00:00|4599-12-31 00:00:00|
|     1008|   Loans|small business loan|1980-01-01 00:00:00|4599-12-31 00:00:00|
|     1009|   Loans|    home loan extra|2016-12-04 00:00:00|4599-12-31 00:00:00|
|     1010|   Loans| persona

In [22]:
spark_df5 = spark_df5.withColumn('Start Date', date_format(to_date(col('Start Date')), 'yyyy-MM-dd')).withColumn('End Date', date_format(to_date(col('End Date')), 'yyyy-MM-dd'))

In [23]:
spark_df5 = spark_df5.withColumn('Start Date', to_date(col('Start Date'))).withColumn('End Date', to_date(col('End Date')))

In [24]:
spark_df4.write.csv('Barclays/DataTrekAssessment', header=True)

In [25]:
spark_df5.write.csv('Barclays/DataTrekAssessment/5', header=True)

In [26]:
spark_df3.write.csv('Barclays/DataTrekAssessment/3', header=True)

24/07/11 15:13:59 WARN TaskSetManager: Stage 10 contains a task of very large size (1622 KiB). The maximum recommended task size is 1000 KiB.


In [27]:
spark_df1.write.csv('Barclays/DataTrekAssessment/1', header=True)

In [28]:
spark_df2.write.csv('Barclays/DataTrekAssessment/2', header=True)

24/07/11 15:14:00 WARN TaskSetManager: Stage 12 contains a task of very large size (3278 KiB). The maximum recommended task size is 1000 KiB.


In [29]:
pandasDF1 = spark_df1.toPandas()

In [30]:
pandasDF2 = spark_df2.toPandas()
pandasDF3 = spark_df3.toPandas()
pandasDF4 = spark_df4.toPandas()
pandasDF5 = spark_df5.toPandas()

24/07/11 15:14:01 WARN TaskSetManager: Stage 14 contains a task of very large size (3278 KiB). The maximum recommended task size is 1000 KiB.
24/07/11 15:14:05 WARN TaskSetManager: Stage 15 contains a task of very large size (1622 KiB). The maximum recommended task size is 1000 KiB.


In [31]:
pandasDF1.to_csv('Customer_Product_modified.csv', header=True, index=False)
pandasDF2.to_csv('Customer_Transaction_history_modified.csv', header=True, index=False)
pandasDF3.to_csv('Customer_Channel_Activity_modified.csv', header=True, index=False)
pandasDF4.to_csv('customer_demographic_modified.csv', header=True, index=False)
pandasDF5.to_csv('Product_Lookup_modified.csv', header=True, index=False)